<a href="https://colab.research.google.com/github/sarahjlassi/AG_TSP/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

POPULATION_SIZE = 5
NB_GENES = 15
MUTATION_RATE = 0.2
CROSSING_RATE = 0.7

TOURNAMENT_SELECTION_SIZE = 4

Villes = []
i=0
while i < NB_GENES:
  ville = (random.randint(0,9),random.randint(0,9))
  if ville not in Villes:
    Villes.append(ville)
    i += 1

In [ ]:
Villes

[(3, 0), (5, 4), (3, 2), (3, 5), (2, 3)]

In [2]:
class Chromosome:
    '''Chromosome Init'''
    def __init__(self, villes):
        self.genes = []
        self.fitness = 0
        i = 0
        while i < NB_GENES:
          ville = Villes[random.randint(0,NB_GENES - 1)]
          if ville not in self.genes:
            self.genes.append(ville)
            i += 1
         
    def get_genes(self):
        return self.genes

    def get_X(self,pos):
        return self.genes[pos][0]

    def get_Y(self, pos):
        return self.genes[pos][1]

    def get_fitness(self):
        self.fitness = 0

        for i in range(NB_GENES-1):
          #distance = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                # self.fitness += math.sqrt((self.genes[i][0] - self.genes[i+1][0])**2 + (self.genes[i][1] - self.genes[i+1][1])**2)

                self.fitness += math.sqrt((self.get_X(i) - self.get_X(i+1))**2 + (self.get_Y(i) - self.get_Y(i+1))**2)
        self.fitness += math.sqrt((self.get_X(NB_GENES-1) - self.get_X(0))**2 + (self.get_Y(NB_GENES-1) - self.get_Y(0))**2)       
        return round(self.fitness, 2) 

    def __str__(self):
        return self.genes.__str__()

In [ ]:
Chromosome(Villes).__str__()

'[(3, 0), (2, 3), (3, 2), (3, 5), (5, 4)]'

In [3]:
class Population:
    '''Population Init'''
    def __init__(self, size):
        self.chromosomes = []
        i = 0
        while i < size :
            self.chromosomes.append( Chromosome(Villes) )
            i += 1
        self.chromosomes.sort(key=lambda x: x.get_fitness(), reverse=False)

    '''Get All Population Chromosomes'''
    def get_chromosomes(self):
        return self.chromosomes

    def print_population(self, gen_number):
        print("\n-----------------------Generation Summary---------------------------")
        print("--------------------------------------------------------------------")
        print("Generation #", gen_number, "| Fittest chromosome fitness:", self.get_chromosomes()[0].get_fitness())
        print("--------------------------------------------------------------------")
        i = 0
        for x in self.get_chromosomes():
            print("Chromosome #",i," :",x,"| Fitness", x.get_fitness())
            i += 1

In [ ]:
Population(5).print_population(0)


-----------------------Generation Summary---------------------------
--------------------------------------------------------------------
Generation # 0 | Fittest chromosome fitness: 12.46
--------------------------------------------------------------------
Chromosome # 0  : [(5, 4), (3, 2), (3, 0), (2, 3), (3, 5)] | Fitness 12.46
Chromosome # 1  : [(3, 2), (3, 0), (2, 3), (5, 4), (3, 5)] | Fitness 13.56
Chromosome # 2  : [(3, 0), (5, 4), (3, 2), (3, 5), (2, 3)] | Fitness 15.7
Chromosome # 3  : [(3, 2), (5, 4), (3, 0), (2, 3), (3, 5)] | Fitness 15.7
Chromosome # 4  : [(3, 5), (3, 0), (5, 4), (2, 3), (3, 2)] | Fitness 17.05


In [4]:
def select_tournament(pop):
    tournament_pop = Population(0)
    i = 0
    while i < TOURNAMENT_SELECTION_SIZE :
        tournament_pop.get_chromosomes().append(pop.get_chromosomes()[random.randrange(0,POPULATION_SIZE)])
        i += 1
    tournament_pop.get_chromosomes().sort(key=lambda x: x.get_fitness(), reverse=False)
    return tournament_pop.get_chromosomes()[0]

In [5]:
def crossover_chromosomes(parent1, parent2):
    if random.random() < CROSSING_RATE:
      villes = []
      child1 = Chromosome(villes)
      child2 = Chromosome(villes)

      '''One Point Cross Over'''
      index = random.randrange(1, NB_GENES)
      sub_1_child1 = parent1.get_genes()[:index]
      sub_2_child1 = [item for item in parent2.get_genes() if item not in sub_1_child1]
      child1.genes = sub_1_child1 + sub_2_child1
      sub_1_child2 = parent2.get_genes()[:index]
      sub_2_child2 = [item for item in parent1.get_genes() if item not in sub_1_child2]
      child2.genes = sub_1_child2 + sub_2_child2


      print("\nMaking a cross")
      print("Parent1: ",parent1.get_genes())
      print("Parent2: ",parent2.get_genes())
      print("Child1 : ", child1.get_genes())
      print("Child2 : ", child2.get_genes())

      return child1, child2

    else:
      return parent1, parent2

In [8]:
def mutate(individual):
    mutate_index1 = random.randrange(len(individual.get_genes()))
    mutate_index2 = random.randrange(len(individual.get_genes()))
    individual.get_genes()[mutate_index1], individual.get_genes()[mutate_index2]= individual.get_genes()[mutate_index2], individual.get_genes()[mutate_index1]

In [9]:
def evolve(pop):
    new_pop = Population(0)
    print("\nCrossover and Mutation Trace:")
    while new_pop.get_chromosomes().__len__() < POPULATION_SIZE:
        parent1 = select_tournament(pop)
        parent2 = select_tournament(pop)

        child1, child2 = crossover_chromosomes(parent1, parent2)

        mutate(child1)
        mutate(child2)

        new_pop.get_chromosomes().append(child1)

        if len(new_pop.get_chromosomes()) < POPULATION_SIZE:
            new_pop.get_chromosomes().append(child2)

    new_pop.get_chromosomes().sort(key=lambda x: x.get_fitness(), reverse=False)   
    return new_pop

In [10]:
generation_number = 0
MAX_FITNESS = 10
population = Population(POPULATION_SIZE)
population.print_population(generation_number)

nb = 0
while population.get_chromosomes()[0].get_fitness() > MAX_FITNESS and nb < 3 :
  nb = nb + 1
  generation_number += 1
  population = evolve(population)
  population.print_population(generation_number)


-----------------------Generation Summary---------------------------
--------------------------------------------------------------------
Generation # 0 | Fittest chromosome fitness: 71.39
--------------------------------------------------------------------
Chromosome # 0  : [(7, 5), (8, 6), (4, 7), (0, 2), (1, 9), (2, 7), (3, 2), (1, 8), (6, 7), (2, 3), (4, 4), (9, 1), (7, 1), (0, 4), (7, 2)] | Fitness 71.39
Chromosome # 1  : [(2, 3), (7, 2), (7, 5), (1, 9), (9, 1), (7, 1), (2, 7), (1, 8), (4, 7), (3, 2), (4, 4), (0, 4), (8, 6), (6, 7), (0, 2)] | Fitness 72.87
Chromosome # 2  : [(0, 4), (1, 8), (7, 5), (0, 2), (7, 2), (9, 1), (2, 7), (8, 6), (3, 2), (4, 7), (6, 7), (7, 1), (4, 4), (1, 9), (2, 3)] | Fitness 80.96
Chromosome # 3  : [(0, 2), (2, 7), (8, 6), (2, 3), (7, 1), (1, 9), (1, 8), (7, 5), (0, 4), (4, 4), (4, 7), (9, 1), (6, 7), (7, 2), (3, 2)] | Fitness 81.96
Chromosome # 4  : [(2, 7), (7, 5), (6, 7), (0, 2), (7, 2), (0, 4), (3, 2), (8, 6), (4, 4), (2, 3), (7, 1), (1, 8), (9, 1)